# Read Image data into memory

In [7]:
import time
import getpass
import warnings
warnings.filterwarnings('ignore')

username = getpass.getuser()
start_time = time.time()
#Where is the data?
if username == "bdutt":
    baseDir="/Users/bdutt/Documents/workspace-whitespace/face/face/data/lfw"
elif username =="cgarg":
    baseDir="/Users/cgarg/Documents/charu/intuit/code/face/data/lfw"
flow = "main"
debug = False

%run "Load Data.ipynb"

filenames = []
labels = []

def collectFileNamesAndLabels(filename):
    filenames.append(filename)
    result = filename.split("/") 
    labels.append(result[len(result)-2]) #Get the label name from filename
    

doForEachFile(collectFileNamesAndLabels,baseDir, 10)
print("Total File name collection time = %s seconds" % (time.time() - start_time))
print ("Files collected : ",len(filenames))

Total File name collection time = 0.017796993255615234 seconds
Files collected :  6


In [8]:
start_time = time.time()
imagesList = []
doForEachFileNames(readImage, filenames, imagesList)
print ("Images data collected : ",len(imagesList))
print("Total data collection time = %s seconds" % (time.time() - start_time))

Images data collected :  6
Total data collection time = 0.017261028289794922 seconds


In [12]:
%run "TransferLearningWithCNN.ipynb"

comic_book (100.00%)


{'Conv2d_1a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_1a_3x3/Relu:0' shape=(?, 149, 149, 32) dtype=float32>, 'Conv2d_2a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_2a_3x3/Relu:0' shape=(?, 147, 147, 32) dtype=float32>, 'Conv2d_2b_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_2b_3x3/Relu:0' shape=(?, 147, 147, 64) dtype=float32>, 'MaxPool_3a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/MaxPool_3a_3x3/MaxPool:0' shape=(?, 73, 73, 64) dtype=float32>, 'Conv2d_3b_1x1': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_3b_1x1/Relu:0' shape=(?, 73, 73, 80) dtype=float32>, 'Conv2d_4a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_4a_3x3/Relu:0' shape=(?, 71, 71, 192) dtype=float32>, 'MaxPool_5a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/MaxPool_5a_3x3/MaxPool:0' shape=(?, 35, 35, 192) dtype=float32>, 'Mixed_5b': <tf.Tensor 'InceptionV3/InceptionV3/Mixed_5b/concat:0' shape=(?, 35, 35, 256) dtype=float32>, 'Mixed_5c': <tf.Tensor 'InceptionV3/InceptionV3/Mixed_5c/concat:0' shape=(?, 35, 35,

NameError: name 'class_images' is not defined

# Generic Multi threading code

Used later

In [3]:
import threading
import sys
import traceback
import time

class myThread (threading.Thread):
    def __init__(self, threadName, index, worker):
        threading.Thread.__init__(self)
        self.threadName = threadName
        self.index = index
        self.worker = worker
    def run(self):
        print('^',self.threadName, ', ', end='')
        try:
            self.worker(self.threadName, self.index)
        except:
            e = sys.exc_info()[0]
            print("Exception in thread: ",self.threadName,", ", e)
            traceback.print_exc()
        print('V',self.threadName, end='')
        

def runThreads(nThreads, worker):
    threads = []
    for i in range(nThreads):
        try:
            t = myThread("Thread-"+str(i), i , worker)
            threads.append(t)
            t.start()
        except:
            e = sys.exc_info()[0]
            print("Error: unable to start thread: ", e)
            traceback.print_exc()

    time.sleep(1)

    for t in threads:
        t.join()

In [4]:
start_time = time.time()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])


Y = pd.DataFrame(labels, columns = ['name'])
sizeOfYLabel = len(Y)
oneDImageList = []
showOneImage = False

imageListLock = threading.Lock()

def flattenImage(image, imageList):
    # Convert RGB to single number
    global showOneImage
    if showOneImage:
        plt.imshow(image, cmap="Greys")
        plt.show()
    gray = rgb2gray(image)
    if showOneImage:
        plt.imshow(gray, cmap="Greys")
        plt.show()
        showOneImage = False
    imageList.append(gray.flatten())

nThreads = 10
splitImagesList = np.array_split(imagesList, nThreads)
splitLabelsList = np.array_split(labels, nThreads)
labels = []

def imageFlatteningWorker(threadName, index):
    global splitImagesList
    global imageListLock
    imageList = []
    for image in splitImagesList[index]:
        flattenImage(image, imageList)
    imageListLock.acquire()
    global oneDImageList
    oneDImageList.extend(imageList)
    global labels
    labels.extend(splitLabelsList[index])
    imageListLock.release()

runThreads(nThreads, imageFlatteningWorker)
    
del imagesList

X = pd.DataFrame(oneDImageList)
del oneDImageList
print("X shape", X.shape)
print("label count", len(labels))
print("Total dataframe creation time = %s seconds" % (time.time() - start_time))

^ Thread-0 , ^ Thread-1 , ^ Thread-2 , ^^ Thread-3 ,  Thread-4 , ^ Thread-5 , ^ Thread-6 , ^^ Thread-7 , ^ Thread-8 ,  Thread-9 , V Thread-3V Thread-0V Thread-1V Thread-8V Thread-2V Thread-5V Thread-6VV Thread-9 Thread-7V Thread-4X shape (647, 62500)
label count 647
Total dataframe creation time = 11.837629795074463 seconds


In [5]:
#Y['name'].value_counts()

In [6]:
start_time = time.time()
maxFrequency = 21

# We upsample/downsample to have equal quantity of all classes

#X_sample = pd.DataFrame(columns = X.columns)
#Y_sample = pd.DataFrame(columns = Y.columns)
X_sampleList = []
Y_sampleList = []

XY_sampleLock = threading.Lock()

def equalSampleLabel(label):
    global X_sampleList
    global Y_sampleList
    global XY_sampleLock
    dfa = X[Y['name']== label]
    dfasample = dfa.sample(n=maxFrequency,replace=True)
    dfb = Y[Y['name']== label]
    dfbsample = dfb.sample(n=maxFrequency,replace=True)
    print('.', end='')
    XY_sampleLock.acquire()
    print('-', end='') 
    X_sampleList.append(dfasample)
    Y_sampleList.append(dfbsample)
    XY_sampleLock.release()
    print('+', end='')



labels = Y['name'].unique()

nThreads = 10
labelsArray = np.array_split(labels, nThreads)

def worker(threadName, index):
    for label in labelsArray[index]:
        #print(threadName, label)
        equalSampleLabel(label)

runThreads(nThreads, worker)

X_sample = pd.concat(X_sampleList)
Y_sample = pd.concat(Y_sampleList)
print(X_sample.shape)
print(Y_sample.shape)
X = X_sample
Y = Y_sample
# print(X_sample)
print("Total equal sampling time = %s seconds" % (time.time() - start_time))

^ Thread-0 , ^ Thread-1 , ^ Thread-2 , ^ Thread-3 , ^ Thread-4 , ^ Thread-5 , ^ Thread-6 , ^ Thread-7 , .-+^ Thread-8 , .-.^ Thread-9 , .+-+-+.-+.-+...-+-+-+.-+.-+.-+.-+..-+.-+-.+-+..-+-+.-+.-+..-+-+...-+-+-+.-+....-+-+-+.-+-+.-+.-.+-+..-.+..-+-+-+-+.-+..-+.-.+-...+-+-...+-+-+.-+-..+-+.-+-+-+-+.-+..-+-+..-+-+...-+-+-.+..-+.-+-+-+.-+..-+-+..-+-+.-+.-+.-+.-+.-+..-+-..+-+..-+.-+.-+-+-+.-+..-+-.+....-+-+..-+-+-+-...+-+-+-.+-+.-+-+.-+.-+.-+..-+-+.-+.-+.-+.-+.-+.-+.-+.-+..-+..-+-+-....+-+-+-+-+..-+-+.-+.-+.-+.-+...-+-+-+.-+.-+..-.+.-+.-+-+-+.-...+-...+.-.+-+-+..-+-+-.+-+-+-+.-..+-+.-+-..+-+-+-+.-+..-.+.-+.-..+-.+-+-..+.-+-.+.-.+-+-+-+.-+-+-+.-+.-+.-+..-....+.-+-+-+-.+-+-+.-+-+.-+..-+-+.-+.-.+.-.+..-.+-+...-+-+-+-.+-+-+-..+-.+-+-...+...-+-+-+-.+-.+.-+-+-.+.-+.-+-+-+.-+.-+.-+.-+..-+-+..-+..-+-+-+..-+-+.-+.-.+.-+-+..-+-.+-+.-+.-+.-+.-+.-+.-+.-+...-+-+-+.-+.-+..-+-.+.-+.-+-+.-+.-+.-+.-+.-+..-+.-+-.+-.+..-.+-+-+..-+-+-+.-+.-+.-+.-.+-+..-..+.-+.-+.-+-+.-+-+-+.-.+-+..-+.-+.-+.-+.-+-+.-+.-+..-+-+.-+

In [7]:
start_time = time.time()
sizeOfX = X_sample.shape[1]
%run "DNN.ipynb"

print("Total DNN time = %s seconds" % (time.time() - start_time))

Shape of X to DNN (7455, 62500)
Shape of Y to DNN (7455, 1)


Using TensorFlow backend.


Shape of Y post OneHot encoding (7455, 355)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                3125050   
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 355)               9230      
Total params: 3,135,555
Trainable params: 3,135,555
Non-trainable params: 0
_________________________________________________________________
Epoch

Epoch 117/500
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0195 - categorical_accuracy: 0.0919
Epoch 118/500
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0193 - categorical_accuracy: 0.0930
Epoch 119/500
7455/7455 [==============================] - 20s 3ms/step - loss: 0.0192 - categorical_accuracy: 0.0997
Epoch 120/500
7455/7455 [==============================] - 18s 2ms/step - loss: 0.0192 - categorical_accuracy: 0.0987
Epoch 121/500
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0192 - categorical_accuracy: 0.0959
Epoch 122/500
7455/7455 [==============================] - 20s 3ms/step - loss: 0.0191 - categorical_accuracy: 0.1017
Epoch 123/500
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0191 - categorical_accuracy: 0.0963
Epoch 124/500
7455/7455 [==============================] - 20s 3ms/step - loss: 0.0190 - categorical_accuracy: 0.1057
Epoch 125/500
7455/7455 [==============================]

7455/7455 [==============================] - 21s 3ms/step - loss: 0.0141 - categorical_accuracy: 0.3584
Epoch 256/500
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0140 - categorical_accuracy: 0.3646
Epoch 257/500
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0141 - categorical_accuracy: 0.3606
Epoch 258/500
7455/7455 [==============================] - 20s 3ms/step - loss: 0.0139 - categorical_accuracy: 0.3771
Epoch 259/500
7455/7455 [==============================] - 14s 2ms/step - loss: 0.0140 - categorical_accuracy: 0.3636
Epoch 260/500
7455/7455 [==============================] - 19s 3ms/step - loss: 0.0139 - categorical_accuracy: 0.3682
Epoch 261/500
7455/7455 [==============================] - 20s 3ms/step - loss: 0.0138 - categorical_accuracy: 0.3768
Epoch 262/500
7455/7455 [==============================] - 16s 2ms/step - loss: 0.0138 - categorical_accuracy: 0.3764
Epoch 263/500
7455/7455 [==============================] - 20s 3ms/ste

7455/7455 [==============================] - 15s 2ms/step - loss: 0.0105 - categorical_accuracy: 0.5910
Epoch 394/500
7455/7455 [==============================] - 18s 2ms/step - loss: 0.0104 - categorical_accuracy: 0.6011
Epoch 395/500
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0105 - categorical_accuracy: 0.5981
Epoch 396/500
7455/7455 [==============================] - 16s 2ms/step - loss: 0.0105 - categorical_accuracy: 0.5988
Epoch 397/500
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0105 - categorical_accuracy: 0.5956
Epoch 398/500
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0105 - categorical_accuracy: 0.5977
Epoch 399/500
7455/7455 [==============================] - 21s 3ms/step - loss: 0.0105 - categorical_accuracy: 0.6005
Epoch 400/500
7455/7455 [==============================] - 20s 3ms/step - loss: 0.0104 - categorical_accuracy: 0.5993
Epoch 401/500
7455/7455 [==============================] - 21s 3ms/ste

In [8]:
# print(Y)
enc.get_feature_names(['name'])
print(enc.get_feature_names(['name']).shape)

(355,)


In [9]:
import numpy as np
import tensorflow as tf

Ypred = pd.DataFrame( columns = ['sampleId', "classAsIndex", "classAsString", "actualClass"])
for i in range(300):
    prediction = model.predict(X_sample.iloc[i:i+1,:],verbose=0)
    class_labels = np.argmax(prediction[0], axis=0)
    Ypred.at[i, 'sampleId']  = i
    Ypred.at[i, 'classAsIndex']  = class_labels
    Ypred.at[i, 'classAsString']  = enc.get_feature_names(['name'])[class_labels]
    Ypred.at[i, 'actualClass']  = Y_sample.iloc[i].values
    #print(class_labels, enc.get_feature_names(['name'])[class_labels]," Names of class :: ", Y_sample.iloc[i].values)

from IPython.display import display, HTML
display(HTML(Ypred.to_html()))

,sampleId,classAsIndex,classAsString,actualClass
0,0,116,name_German_Khan,[German_Khan]
1,1,116,name_German_Khan,[German_Khan]
2,2,116,name_German_Khan,[German_Khan]
3,3,116,name_German_Khan,[German_Khan]
4,4,116,name_German_Khan,[German_Khan]
5,5,116,name_German_Khan,[German_Khan]
6,6,116,name_German_Khan,[German_Khan]
7,7,116,name_German_Khan,[German_Khan]
8,8,116,name_German_Khan,[German_Khan]
9,9,116,name_German_Khan,[German_Khan]
